In [ ]:
!pip install onnx

In [3]:
import onnx
from onnx import version_converter, helper as h


In [10]:

# Preprocessing: load the model to be converted.
model_path = "../artifacts/mosaic_original.onnx"
original_model = onnx.load(model_path)

original_model_opset_version = original_model.opset_import[0].version
target_model_opset_version = 17

print(f"Original opset_version: {original_model_opset_version}")
print(f"Target opset_version: {target_model_opset_version}")

print("Start conversion ...")


# A full list of supported adapters can be found here:
# https://github.com/onnx/onnx/blob/main/onnx/version_converter.py#L21
# Apply the version conversion on the original model
converted_model = version_converter.convert_version(original_model, target_model_opset_version)


model_graph = converted_model.graph


print("Fixing Resize nodes...")

new_nodes = []
for i, node in enumerate(model_graph.node):
    if node.op_type == "Resize":
        new_resize = onnx.helper.make_node(
            'Resize',
            inputs=node.input,
            outputs=node.output,
            name=node.name,
            mode='linear'
        )
        # Update node
        new_nodes += [new_resize]
    else:
        new_nodes += [node]

new_graph = h.make_graph(
    new_nodes,
    model_graph.name,
    model_graph.input,
    model_graph.output,
    initializer=model_graph.initializer,  # The initializer holds all non-constant weights.
)

new_model = h.make_model(new_graph, producer_name="onnx-fix-nodes")
new_model.opset_import[0].version = converted_model.opset_import[0].version

print("Save new model")
onnx.save(new_model, f"../artifacts/mosaic.onnx")



Original opset_version: 9
Target opset_version: 17
Start conversion ...
Fixing Resize nodes...
Save new model
